In [ ]:
! pip install taichi
! pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.0/712.0 kB 19.6 MB/s eta 0:00:00


In [ ]:
import taichi as ti
import os
import trimesh
import numpy as np
from IPython.display import Image, display

ti.init(arch=ti.cuda) # Alternatively, ti.init(arch=ti.cpu)

[Taichi] Starting on arch=cuda


In [ ]:
# remove existing results
! mkdir "output"
output_path = 'output'

for filename in os.listdir(output_path):
    file_path = os.path.join(output_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)

## Initialise mesh indices and positions in 3D

In [ ]:

@ti.kernel
def initialize_mesh_indices():
    for i, j in ti.ndrange(n - 1, n - 1):
        quad_id = (i * (n - 1)) + j
        # 1st triangle of the square
        indices[quad_id * 6 + 0] = i * n + j
        indices[quad_id * 6 + 1] = (i + 1) * n + j
        indices[quad_id * 6 + 2] = i * n + (j + 1)

        # 2nd triangle of the square
        indices[quad_id * 6 + 3] = (i + 1) * n + j + 1
        indices[quad_id * 6 + 4] = i * n + (j + 1)
        indices[quad_id * 6 + 5] = (i + 1) * n + j




In [ ]:

@ti.kernel
def initialize_mass_points():
    random_offset = ti.Vector([ti.random() - 0.5, ti.random() - 0.5]) * 0.1

    for i, j in x:
        x[i, j] = [
            i * quad_size - 0.5 + random_offset[0], 0.5,
            j * quad_size - 0.5 + random_offset[1]
        ]
        v[i, j] = [0, 0, 0]

In [ ]:

n = 128  # Number of mass points per side (cloth resolution)
quad_size = 1.0 / n  # Rest length between adjacent mass points
dt = 4e-2 / n  # Time step size
substeps = int(1 / 60 // dt)  # Number of substeps per frame for stability


# Fields for positions and velocities of mass points
x = ti.Vector.field(3, dtype=float, shape=(n, n))
v = ti.Vector.field(3, dtype=float, shape=(n, n))


# Mesh data for rendering
num_triangles = (n - 1) * (n - 1) * 2
indices = ti.field(int, shape=num_triangles * 3)
vertices = ti.Vector.field(3, dtype=float, shape=n*n)

ball_radius = 0.3
ball_center = ti.Vector.field(3, dtype=float, shape=(1, ))
ball_center[0] = [0, 0, 0]

# Initialize mesh indices for rendering triangles
initialize_mesh_indices()
initialize_mass_points() # Initialize the cloth grid


In [ ]:
# Simulation parameters

# Physical constants
gravity = ti.Vector([0, -9.8, 0])  # Gravity vector
spring_Y = 3e6  # Spring stiffness (Young's modulus)
dashpot_damping = 4e4 # Damping coefficient for springs

bending_springs = False # Toggle for bending springs (not used here)



# Define spring connections (structural and shear springs)
spring_offsets = []
if bending_springs:
    # Bending springs: connect to all 8 neighbors
    for i in range(-1, 2):
        for j in range(-1, 2):
            if (i, j) != (0, 0):
                spring_offsets.append(ti.Vector([i, j]))

else:
    # Structural and shear springs: connect to neighbors within Manhattan distance 2
    for i in range(-2, 3):
        for j in range(-2, 3):
            if (i, j) != (0, 0) and abs(i) + abs(j) <= 2:
                spring_offsets.append(ti.Vector([i, j]))



In [ ]:

@ti.kernel
def substep():
    # Apply gravity to all mass points

    for i in ti.grouped(x):
        v[i] += gravity * dt   #gravity force update

    # Compute spring and damping forces
    for i in ti.grouped(x):
        force = ti.Vector([0.0, 0.0, 0.0])
        # Pin the top corners of the cloth
        if((i[0]==n-1 and i[1]==n-1) or (i[0]==0 and i[1]==n-1) ):
            pass

        else:
            for spring_offset in ti.static(spring_offsets):
                j = i + spring_offset
                if 0 <= j[0] < n and 0 <= j[1] < n:
                    x_ij = x[i] - x[j]
                    v_ij = v[i] - v[j]
                    dir = x_ij.normalized()
                    current_dist = x_ij.norm()
                    original_dist = quad_size * float(i - j).norm()

                    # Hooke's law: F = -k * (stretch ratio - 1) * direction
                    force += -spring_Y * dir * ( current_dist - original_dist)


            v[i] += force * dt  # update velocity
    # Update positions
    for i in ti.grouped(x):
        offset_to_center = x[i] - ball_center[0]
        if offset_to_center.norm() <= ball_radius:
            # Velocity projection
            normal = offset_to_center.normalized()
            v[i] -= min(v[i].dot(normal), 0) * normal



        x[i] += dt * v[i]

In [ ]:
@ti.kernel
def update_vertices():
    # Flatten the 2D grid of positions into a 1D array for rendering

    for i, j in ti.ndrange(n, n):
        vertices[i * n + j] = x[i, j]


In [ ]:
# Set up the Taichi GUI window and scene
window = ti.ui.Window("Taichi Cloth Simulation on GGUI", (1024, 1024), vsync=True,show_window=False)
canvas = window.get_canvas()
canvas.set_background_color((1, 1, 1))
scene = ti.ui.Scene()
camera = ti.ui.Camera()
video_manager = ti.tools.VideoManager(output_dir=output_path, framerate=60, automatic_build=False)

current_t = 0.0
initialize_mass_points()
frame=0
secs = 1.5
while frame<secs*60:
    if current_t > 1.5:
        # Reset
        initialize_mass_points()
        current_t = 0

    for i in range(substeps):
        substep()
        current_t += dt
    update_vertices()

    camera.position(0.0, 0.0, 3)
    camera.lookat(0.0, 0.0, 0)
    scene.set_camera(camera)

    scene.point_light(pos=(0, 1, 2), color=(1, 1, 1))
    scene.ambient_light((0.5, 0.5, 0.5))
    scene.mesh(vertices,
            indices=indices,show_wireframe=True,
            two_sided=True)

    # Draw a smaller ball to avoid visual penetration
    scene.particles(ball_center, radius=ball_radius * 0.95, color=(0.5, 0.42, 0.8))
    canvas.scene(scene)
    img = window.get_image_buffer_as_numpy()
    video_manager.write_frame(img)
    frame+=1
video_manager.make_video(gif=True, mp4=False)


RuntimeError: Vulkan must be available for GGUI

In [ ]:
display(Image(data=open(os.path.join(output_path,'video.gif'),'rb').read(), format='png'))


FileNotFoundError: [Errno 2] No such file or directory: 'output/video.gif'